In [6]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 24: Crossed Wires ---</h2><p>You and The Historians arrive at the edge of a <a href="/2022/day/23">large grove</a> somewhere in the jungle. After the last incident, the Elves installed a small device that monitors the fruit. While The Historians search the grove, one of them asks if you can take a look at the monitoring device; apparently, it's been malfunctioning recently.</p>
<p>The device seems to be trying to produce a number through some boolean logic gates. Each gate has two inputs and one output. The gates all operate on values that are either <em>true</em> (<code>1</code>) or <em>false</em> (<code>0</code>).</p>
<ul>
<li><code>AND</code> gates output <code>1</code> if <em>both</em> inputs are <code>1</code>; if either input is <code>0</code>, these gates output <code>0</code>.</li>
<li><code>OR</code> gates output <code>1</code> if <em>one or both</em> inputs is <code>1</code>; if both inputs are <code>0</code>, these gates output <code>0</code>.</li>
<li><code>XOR</code> gates output <code>1</code> if the inputs are <em>different</em>; if the inputs are the same, these gates output <code>0</code>.</li>
</ul>
<p>Gates wait until both inputs are received before producing output; wires can carry <code>0</code>, <code>1</code> or no value at all. There are no loops; once a gate has determined its output, the output will not change until the whole system is reset. Each wire is connected to at most one gate output, but can be connected to many gate inputs.</p>
<p>Rather than risk getting shocked while tinkering with the live system, you write down all of the gate connections and initial wire values (your puzzle input) so you can consider them in relative safety. For example:</p>
<pre><code>x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -&gt; z00
x01 XOR y01 -&gt; z01
x02 OR y02 -&gt; z02
</code></pre>

<p>Because gates wait for input, some wires need to start with a value (as inputs to the entire system). The first section specifies these values. For example, <code>x00: 1</code> means that the wire named <code>x00</code> starts with the value <code>1</code> (as if a gate is already outputting that value onto that wire).</p>
<p>The second section lists all of the gates and the wires connected to them. For example, <code>x00 AND y00 -&gt; z00</code> describes an instance of an <code>AND</code> gate which has wires <code>x00</code> and <code>y00</code> connected to its inputs and which will write its output to wire <code>z00</code>.</p>
<p>In this example, simulating these gates eventually causes <code>0</code> to appear on wire <code>z00</code>, <code>0</code> to appear on wire <code>z01</code>, and <code>1</code> to appear on wire <code>z02</code>.</p>
<p>Ultimately, the system is trying to produce a <em>number</em> by combining the bits on all wires starting with <code>z</code>. <code>z00</code> is the least significant bit, then <code>z01</code>, then <code>z02</code>, and so on.</p>
<p>In this example, the three output bits form the binary number <code>100</code> which is equal to the decimal number <code><em>4</em></code>.</p>
<p>Here's a larger example:</p>
<pre><code>x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -&gt; mjb
y02 OR x01 -&gt; tnw
kwq OR kpj -&gt; z05
x00 OR x03 -&gt; fst
tgd XOR rvg -&gt; z01
vdt OR tnw -&gt; bfw
bfw AND frj -&gt; z10
ffh OR nrd -&gt; bqk
y00 AND y03 -&gt; djm
y03 OR y00 -&gt; psh
bqk OR frj -&gt; z08
tnw OR fst -&gt; frj
gnj AND tgd -&gt; z11
bfw XOR mjb -&gt; z00
x03 OR x00 -&gt; vdt
gnj AND wpb -&gt; z02
x04 AND y00 -&gt; kjc
djm OR pbm -&gt; qhw
nrd AND vdt -&gt; hwm
kjc AND fst -&gt; rvg
y04 OR y02 -&gt; fgs
y01 AND x02 -&gt; pbm
ntg OR kjc -&gt; kwq
psh XOR fgs -&gt; tgd
qhw XOR tgd -&gt; z09
pbm OR djm -&gt; kpj
x03 XOR y03 -&gt; ffh
x00 XOR y04 -&gt; ntg
bfw OR bqk -&gt; z06
nrd XOR fgs -&gt; wpb
frj XOR qhw -&gt; z04
bqk OR frj -&gt; z07
y03 OR x01 -&gt; nrd
hwm AND bqk -&gt; z03
tgd XOR rvg -&gt; z12
tnw OR pbm -&gt; gnj
</code></pre>

<p>After waiting for values on all wires starting with <code>z</code>, the wires in this system have the following values:</p>
<pre><code>bfw: 1
bqk: 1
djm: 1
ffh: 0
fgs: 1
frj: 1
fst: 1
gnj: 1
hwm: 1
kjc: 0
kpj: 1
kwq: 0
mjb: 1
nrd: 1
ntg: 0
pbm: 1
psh: 1
qhw: 1
rvg: 0
tgd: 0
tnw: 1
vdt: 1
wpb: 0
z00: 0
z01: 0
z02: 0
z03: 1
z04: 0
z05: 1
z06: 1
z07: 1
z08: 1
z09: 1
z10: 1
z11: 0
z12: 0
</code></pre>
<p>Combining the bits from all wires starting with <code>z</code> produces the binary number <code>0011111101000</code>. Converting this number to decimal produces <code><em>2024</em></code>.</p>
<p>Simulate the system of gates and wires. <em>What decimal number does it output on the wires starting with <code>z</code>?</em></p>
</article>


In [7]:
from operator import and_, or_, xor
from more_itertools import first, last
import re


tests = [
    {
        "name": "Small Example",
        "s": """
            x00: 1
            x01: 1
            x02: 1
            y00: 0
            y01: 1
            y02: 0

            x00 AND y00 -> z00
            x01 XOR y01 -> z01
            x02 OR y02 -> z02
        """,
        "expected": 4,
    },
    {
        "name": "Example",
        "s": """
                x00: 1
                x01: 0
                x02: 1
                x03: 1
                x04: 0
                y00: 1
                y01: 1
                y02: 1
                y03: 1
                y04: 1

                ntg XOR fgs -> mjb
                y02 OR x01 -> tnw
                kwq OR kpj -> z05
                x00 OR x03 -> fst
                tgd XOR rvg -> z01
                vdt OR tnw -> bfw
                bfw AND frj -> z10
                ffh OR nrd -> bqk
                y00 AND y03 -> djm
                y03 OR y00 -> psh
                bqk OR frj -> z08
                tnw OR fst -> frj
                gnj AND tgd -> z11
                bfw XOR mjb -> z00
                x03 OR x00 -> vdt
                gnj AND wpb -> z02
                x04 AND y00 -> kjc
                djm OR pbm -> qhw
                nrd AND vdt -> hwm
                kjc AND fst -> rvg
                y04 OR y02 -> fgs
                y01 AND x02 -> pbm
                ntg OR kjc -> kwq
                psh XOR fgs -> tgd
                qhw XOR tgd -> z09
                pbm OR djm -> kpj
                x03 XOR y03 -> ffh
                x00 XOR y04 -> ntg
                bfw OR bqk -> z06
                nrd XOR fgs -> wpb
                frj XOR qhw -> z04
                bqk OR frj -> z07
                y03 OR x01 -> nrd
                hwm AND bqk -> z03
                tgd XOR rvg -> z12
                tnw OR pbm -> gnj
        """,
        "expected": 2024,
    },
]


def parseI(
    s: str,
) -> tuple[dict[str, int], dict[str, tuple[str, str, str]], dict[str, int | None]]:
    inp, outp = re.split(r"(?:\r?\n){2,}", s.strip())
    values = dict(
        (first(l.strip().split(":")), int(last(l.strip().split(":"))))
        for l in inp.strip().splitlines()
    )

    gates = dict(
        (
            last(map(str.strip, l.strip().split("->"))),
            first(map(str.split, map(str.strip, l.strip().split("->")))),
        )
        for l in outp.strip().splitlines()
    )

    outputs = dict(
        sorted(
            (z, None)
            for l in outp.strip().splitlines()
            if (z := last(map(str.strip, l.strip().split("->")))).startswith("z")
        )
    )

    return values, gates, outputs


def partI(s: str) -> int:
    values, gates, outputs = parseI(s)
    funcs = {"AND": and_, "OR": or_, "XOR": xor}

    while any(v is None for v in outputs.values()):
        for out, (in1, fun, in2) in gates.items():
            v1 = values.get(in1, -1)
            v2 = values.get(in2, -1)

            if v1 != -1 and v2 != -1:
                v_out = funcs[fun](v1, v2)

                if out.startswith("z"):
                    outputs[out] = v_out
                else:
                    values[out] = v_out

    return int("".join(map(str, reversed(outputs.values()))), 2)


@test(tests=tests[:])
def partI_test(s: str) -> int:
    return partI(s)


Test Small Example passed, for partI_test.
Test Example passed, for partI_test.
Success


In [8]:
with open("../input/day24.txt") as f:
    puzzle = f.read()

print(f"Part I: {partI(puzzle)}")

Part I: 59619940979346


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>59619940979346</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>After inspecting the monitoring device more closely, you determine that the system you're simulating is trying to <em>add two binary numbers</em>.</p>
<p>Specifically, it is treating the bits on wires starting with <code>x</code> as one binary number, treating the bits on wires starting with <code>y</code> as a second binary number, and then attempting to add those two numbers together. The output of this operation is produced as a binary number on the wires starting with <code>z</code>. (In all three cases, wire <code>00</code> is the least significant bit, then <code>01</code>, then <code>02</code>, and so on.)</p>
<p>The initial values for the wires in your puzzle input represent <em>just one instance</em> of a pair of numbers that sum to the wrong value. Ultimately, <em>any</em> two binary numbers provided as input should be handled correctly. That is, for any combination of bits on wires starting with <code>x</code> and wires starting with <code>y</code>, the sum of the two numbers those bits represent should be produced as a binary number on the wires starting with <code>z</code>.</p>
<p>For example, if you have an addition system with four <code>x</code> wires, four <code>y</code> wires, and five <code>z</code> wires, you should be able to supply any four-bit number on the <code>x</code> wires, any four-bit number on the <code>y</code> numbers, and eventually find the sum of those two numbers as a five-bit number on the <code>z</code> wires. One of the many ways you could provide numbers to such a system would be to pass <code>11</code> on the <code>x</code> wires (<code>1011</code> in binary) and <code>13</code> on the <code>y</code> wires (<code>1101</code> in binary):</p>
<pre><code>x00: 1
x01: 1
x02: 0
x03: 1
y00: 1
y01: 0
y02: 1
y03: 1
</code></pre>
<p>If the system were working correctly, then after all gates are finished processing, you should find <code>24</code> (<code>11+13</code>) on the <code>z</code> wires as the five-bit binary number <code>11000</code>:</p>
<pre><code>z00: 0
z01: 0
z02: 0
z03: 1
z04: 1
</code></pre>
<p>Unfortunately, your actual system needs to add numbers with many more bits and therefore has many more wires.</p>
<p>Based on <span title="ENHANCE">forensic analysis</span> of scuff marks and scratches on the device, you can tell that there are exactly <em>four</em> pairs of gates whose output wires have been <em>swapped</em>. (A gate can only be in at most one such pair; no gate's output was swapped multiple times.)</p>
<p>For example, the system below is supposed to find the bitwise <code>AND</code> of the six-bit number on <code>x00</code> through <code>x05</code> and the six-bit number on <code>y00</code> through <code>y05</code> and then write the result as a six-bit number on <code>z00</code> through <code>z05</code>:</p>
<pre><code>x00: 0
x01: 1
x02: 0
x03: 1
x04: 0
x05: 1
y00: 0
y01: 0
y02: 1
y03: 1
y04: 0
y05: 1

x00 AND y00 -&gt; z05
x01 AND y01 -&gt; z02
x02 AND y02 -&gt; z01
x03 AND y03 -&gt; z03
x04 AND y04 -&gt; z04
x05 AND y05 -&gt; z00
</code></pre>

<p>However, in this example, two pairs of gates have had their output wires swapped, causing the system to produce wrong answers. The first pair of gates with swapped outputs is <code>x00 AND y00 -&gt; z05</code> and <code>x05 AND y05 -&gt; z00</code>; the second pair of gates is <code>x01 AND y01 -&gt; z02</code> and <code>x02 AND y02 -&gt; z01</code>. Correcting these two swaps results in this system that works as intended for any set of initial values on wires that start with <code>x</code> or <code>y</code>:</p>
<pre><code>x00 AND y00 -&gt; z00
x01 AND y01 -&gt; z01
x02 AND y02 -&gt; z02
x03 AND y03 -&gt; z03
x04 AND y04 -&gt; z04
x05 AND y05 -&gt; z05
</code></pre>
<p>In this example, two pairs of gates have outputs that are involved in a swap. By sorting their output wires' names and joining them with commas, the list of wires involved in swaps is <code><em>z00,z01,z02,z05</em></code>.</p>
<p>Of course, your actual system is much more complex than this, and the gates that need their outputs swapped could be <em>anywhere</em>, not just attached to a wire starting with <code>z</code>. If you were to determine that you need to swap output wires <code>aaa</code> with <code>eee</code>, <code>ooo</code> with <code>z99</code>, <code>bbb</code> with <code>ccc</code>, and <code>aoc</code> with <code>z24</code>, your answer would be <code><em>aaa,aoc,bbb,ccc,eee,ooo,z24,z99</em></code>.</p>
<p>Your system of gates and wires has <em>four</em> pairs of gates which need their output wires swapped - <em>eight</em> wires in total. Determine which four pairs of gates need their outputs swapped so that your system correctly performs addition; <em>what do you get if you sort the names of the eight wires involved in a swap and then join those names with commas?</em></p>
</article>


In [ ]:
import re
from dataclasses import dataclass

from more_itertools import flatten

tests = [
    {
        "name": "Small Example",
        # is not an adder does not a good test.
        "s": """
            x00: 0
            x01: 1
            x02: 0
            x03: 1
            x04: 0
            x05: 1
            y00: 0
            y01: 0
            y02: 1
            y03: 1
            y04: 0
            y05: 1

            x00 AND y00 -> z05
            x01 AND y01 -> z02
            x02 AND y02 -> z01
            x03 AND y03 -> z03
            x04 AND y04 -> z04
            x05 AND y05 -> z00
        """,
        "expected": "z00,z01,z02,z05",
    },
]
# with assistance of claude! Simply too muchwork for a puzzle


@dataclass
class Connection:
    """Represents a logic gate connection."""

    inputs: list[str]
    operation: str
    output: str

    def __str__(self) -> str:
        return f"{self.output} = {self.inputs[0]} {self.operation} {self.inputs[1]}"

    def __eq__(self, other) -> bool:
        return self.inputs == other.inputs and self.operation == other.operation


class Circuit:
    """Represents a logic circuit with gates and wires."""

    OPERATIONS: dict[str, Callable[[int, int], int]] = {
        "OR": lambda x, y: x | y,
        "AND": lambda x, y: x & y,
        "XOR": lambda x, y: x ^ y,
    }

    def __init__(
        self, initial_values: dict[str, int], connections: dict[str, Connection]
    ) -> None:
        """Initialize circuit with initial values and connections."""
        self.initial_values = initial_values.copy()
        self.connections = {
            k: Connection(v.inputs.copy(), v.operation, v.output)
            for k, v in connections.items()
        }
        self._cache: dict[str, int] = {}

    @classmethod
    def from_string(cls, data: str) -> "Circuit":
        """Parse puzzle input and create a Circuit instance."""
        init_pairs = re.findall(r"(.{3}): ([01])", data)
        initial_values = {k: int(v) for k, v in init_pairs}

        connections = {}
        for line in data.split("\n\n")[1].strip().splitlines():
            in1, op, in2, _, out = line.strip().split(" ")
            connections[out] = Connection([in1, in2], op, out)

        return cls(initial_values, connections)

    def evaluate_wire(self, wire: str) -> int:
        """Evaluate a wire's value using memoization."""
        if wire in self._cache:
            return self._cache[wire]

        if wire in self.initial_values:
            return self.initial_values[wire]

        conn = self.connections[wire]
        result = self.OPERATIONS[conn.operation](
            self.evaluate_wire(conn.inputs[0]), self.evaluate_wire(conn.inputs[1])
        )
        self._cache[wire] = result
        return result

    def evaluate_with_inputs(
        self, wire: str, x_values: list[int], y_values: list[int]
    ) -> int:
        """Evaluate a wire with specific x and y input values."""

        def get_value(w: str) -> int:
            match = re.match(r"(x|y)(\d{2})", w)
            if match:
                var, num = match.groups()
                values = x_values if var == "x" else y_values
                return values[int(num)]

            conn = self.connections[w]
            return self.OPERATIONS[conn.operation](
                get_value(conn.inputs[0]), get_value(conn.inputs[1])
            )

        return get_value(wire)

    def swap_wires(self, wire1: str, wire2: str) -> None:
        """Swap the connections of two wires."""
        self.connections[wire1], self.connections[wire2] = (
            self.connections[wire2],
            self.connections[wire1],
        )
        self._cache.clear()

    def find_connection(
        self,
        operation: str | None = None,
        input1: str | None = None,
        input2: str | None = None,
    ) -> Connection | None:
        """Find a connection matching the given criteria."""
        for conn in self.connections.values():
            if operation and conn.operation != operation:
                continue
            if input1 and input1 not in conn.inputs:
                continue
            if input2 and input2 not in conn.inputs:
                continue
            return conn
        return None

    def validate_bit(self, bit_index: int) -> bool:
        """Validate that bit addition works correctly."""
        for x, y, carry in product(range(2), repeat=3):
            x_bits = [0] * (44 - bit_index) + [x]
            y_bits = [0] * (44 - bit_index) + [y]

            if bit_index > 0:
                x_bits += [carry] + [0] * (bit_index - 1)
                y_bits += [carry] + [0] * (bit_index - 1)
            elif carry > 0:
                continue

            x_bits = list(reversed(x_bits))
            y_bits = list(reversed(y_bits))

            z_wire = f"z{bit_index:02d}"
            result = self.evaluate_with_inputs(z_wire, x_bits, y_bits)
            expected = (x + y + carry) % 2

            if result != expected:
                return False
        return True

    @staticmethod
    def make_wire_name(var: str, num: int) -> str:
        """Create a wire name from variable and index."""
        return f"{var}{num:02d}"


class CircuitFixer:
    """Fixes swapped wires in a circuit by identifying and correcting misplaced gates."""

    def __init__(self, circuit: Circuit) -> None:
        """Initialize with a circuit to fix."""
        self.circuit = circuit
        self.swapped_wires: list[str] = []

    def fix_bit(self, bit_index: int) -> list[str]:
        """
        Fix a specific bit in the circuit.

        For a correct adder:
        - zn = nxor XOR m1
        - nxor = xn XOR yn
        - m1 = m2 OR prevand
        - prevand = xn-1 AND yn-1
        - m2 = prevxor AND (something from prev)
        - prevxor = xn-1 XOR yn-1
        """
        print(f"Fixing bit {bit_index}...")

        prev_x = Circuit.make_wire_name("x", bit_index - 1)
        prev_y = Circuit.make_wire_name("y", bit_index - 1)

        prev_and = self.circuit.find_connection(
            operation="AND", input1=prev_x, input2=prev_y
        )
        prev_xor = self.circuit.find_connection(
            operation="XOR", input1=prev_x, input2=prev_y
        )

        m2 = self.circuit.find_connection(operation="AND", input1=prev_xor.output)
        m1 = self.circuit.find_connection(
            operation="OR", input1=m2.output, input2=prev_and.output
        )

        curr_x = Circuit.make_wire_name("x", bit_index)
        curr_y = Circuit.make_wire_name("y", bit_index)
        nxor = self.circuit.find_connection(
            operation="XOR", input1=curr_x, input2=curr_y
        )
        zn = self.circuit.find_connection(
            operation="XOR", input1=nxor.output, input2=m1.output
        )

        to_swap = []

        if zn is None:
            z_wire = Circuit.make_wire_name("z", bit_index)
            zn = self.circuit.connections[z_wire]
            to_swap = list(set(zn.inputs) ^ set([nxor.output, m1.output]))

        if zn.output != Circuit.make_wire_name("z", bit_index):
            to_swap = [Circuit.make_wire_name("z", bit_index), zn.output]

        if to_swap:
            self.circuit.swap_wires(*to_swap)

        return to_swap

    def fix_all(self) -> str:
        """Fix all swapped wires and return sorted comma-separated result."""

        return ",".join(
            sorted(
                flatten(
                    self.fix_bit(bit_index)
                    for bit_index in range(45)
                    if not self.circuit.validate_bit(bit_index)
                )
            )
        )


def partII(puzzle_input: str) -> str:
    """Solve Part II: find and identify all swapped wire pairs."""
    circuit = Circuit.from_string(puzzle_input)
    fixer = CircuitFixer(circuit)
    return fixer.fix_all()


# @test(tests=tests[:])
# def partII_test(s: str) -> int:
#     return partII(s)

In [12]:
print(f"Part II: {partII(puzzle)}")

Fixing bit 6...
Fixing bit 11...
Fixing bit 31...
Fixing bit 38...
Part II: bpt,fkp,krj,mfm,ngr,z06,z11,z31


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>bpt,fkp,krj,mfm,ngr,z06,z11,z31</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
